<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/Stake_SOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratory用 Solanaステーキングプログラム
# 必要なライブラリのインストール
!pip install solana web3
!pip install solders
!pip install solana-py

# ライブラリのインポート
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.rpc.types import TxOpts
from solana.transaction import Transaction
from solana.system_program import TransferParams, transfer
from solana.staking.stake_program import StakeProgram
from solana.staking.instruction import create_stake_account_with_seed_and_delegate_stake, authorize
from solana.staking.state import STAKE_STATE_LEN, StakeAuthorize
from solders.instruction import Instruction

import base58
import time
import json
import os
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd

# Solanaネットワーク接続設定
def setup_client(network="mainnet"):
    """Solanaネットワーククライアントのセットアップ"""
    if network == "mainnet":
        return Client("https://api.mainnet-beta.solana.com")
    elif network == "testnet":
        return Client("https://api.testnet.solana.com")
    elif network == "devnet":
        return Client("https://api.devnet.solana.com")
    else:
        raise ValueError("Invalid network. Choose 'mainnet', 'testnet', or 'devnet'")

# ウォレットキーの読み込み
def load_wallet(key_path=None):
    """ウォレットのキーペアを読み込む"""
    if key_path is None:
        print("秘密鍵ファイルをアップロードしてください（JSON形式、またはBase58エンコードのテキスト）")
        uploaded = files.upload()
        key_path = list(uploaded.keys())[0]

    try:
        # JSONファイルの場合
        with open(key_path, 'r') as f:
            key_data = json.load(f)
            if isinstance(key_data, list):
                # バイトリストからキーペアを作成
                return Keypair.from_bytes(bytes(key_data))
            elif isinstance(key_data, dict) and "privateKey" in key_data:
                # Phantom等のJSONからキーペアを作成
                return Keypair.from_bytes(bytes(key_data["privateKey"]))
    except:
        # Base58エンコードの秘密鍵の場合
        try:
            with open(key_path, 'r') as f:
                secret_key = f.read().strip()
                secret_bytes = base58.b58decode(secret_key)
                return Keypair.from_bytes(secret_bytes)
        except:
            print("サポートされていない形式の鍵ファイルです。")
            return None

# アカウント残高の確認
def check_balance(client, pubkey):
    """アカウントの残高を確認する"""
    response = client.get_balance(pubkey)
    balance_lamports = response["result"]["value"]
    balance_sol = balance_lamports / 10**9  # lamportsからSOLに変換
    return balance_sol

# バリデーターリストの取得
def get_validators(client):
    """現在アクティブなバリデーターのリストを取得"""
    response = client.get_vote_accounts()
    validators = response["result"]["current"]

    # バリデーター情報を整理
    validator_info = []
    for v in validators:
        validator_info.append({
            "identity": v["nodePubkey"],
            "vote_account": v["votePubkey"],
            "commission": v["commission"],
            "active_stake": v["activatedStake"] / 10**9,  # SOL単位
            "last_vote": v["lastVote"]
        })

    # コミッション率でソート
    validator_info = sorted(validator_info, key=lambda x: x["commission"])
    return validator_info

# バリデーターの選択
def select_validator(validators, criteria="commission"):
    """バリデーターを選択する"""
    if criteria == "commission":
        # コミッション率の低いバリデーターを表示
        print("コミッション率の低いバリデーター上位10件:")
        for i, v in enumerate(validators[:10]):
            print(f"{i+1}. コミッション: {v['commission']}%, アクティブステーク: {v['active_stake']:.2f} SOL, ID: {v['identity'][:10]}...")

    selection = int(input("選択するバリデーター番号を入力してください (1-10): "))
    return validators[selection - 1]

# ステーキングアカウントの作成と委任
def create_stake_and_delegate(client, wallet, validator, amount_sol, seed_phrase="stake1"):
    """ステーキングアカウントを作成し、バリデーターに委任する"""
    # SOLをLamportsに変換
    amount_lamports = int(amount_sol * 10**9)

    # ステーキングアカウントの作成
    seed = seed_phrase
    stake_account_pubkey = PublicKey.create_with_seed(
        wallet.public_key, seed, StakeProgram.program_id
    )

    # ステーキングに必要な最小lamports (rent exemption) + stake amount
    required_lamports = client.get_minimum_balance_for_rent_exemption(STAKE_STATE_LEN)["result"]
    required_lamports += amount_lamports

    # バリデーターのvoteアカウント
    vote_account = PublicKey(validator["vote_account"])

    # トランザクションの作成
    tx = Transaction()

    # ステーキングアカウントの作成と委任の命令を追加
    create_stake_instr = StakeProgram.create_account_with_seed(
        {
            "from_pubkey": wallet.public_key,
            "stake_pubkey": stake_account_pubkey,
            "base_pubkey": wallet.public_key,
            "seed": seed,
            "lamports": required_lamports,
            "authorized": {
                "staker": wallet.public_key,
                "withdrawer": wallet.public_key
            }
        }
    )

    delegate_instr = StakeProgram.delegate_stake(
        {
            "stake_pubkey": stake_account_pubkey,
            "authorized_pubkey": wallet.public_key,
            "vote_pubkey": vote_account
        }
    )

    tx.add(create_stake_instr)
    tx.add(delegate_instr)

    # トランザクションの実行
    try:
        tx_sig = client.send_transaction(tx, wallet)
        print(f"トランザクション送信: {tx_sig['result']}")

        # トランザクションの確認を待つ
        for _ in range(30):  # 最大30秒待機
            confirm = client.confirm_transaction(tx_sig["result"])
            if "result" in confirm and confirm["result"]:
                print("ステーキングが完了しました！")
                return stake_account_pubkey, tx_sig["result"]
            time.sleep(1)

        print("トランザクションの確認がタイムアウトしました。Solanaエクスプローラーで確認してください。")
        return stake_account_pubkey, tx_sig["result"]

    except Exception as e:
        print(f"エラー: {str(e)}")
        return None, None

# ステーキング情報の取得
def get_stake_info(client, stake_account):
    """ステーキングアカウントの情報を取得する"""
    response = client.get_stake_activation(stake_account)
    if "result" not in response:
        print("ステーキング情報の取得に失敗しました")
        return None

    stake_info = response["result"]
    print(f"ステーキング状態: {stake_info['state']}")
    print(f"アクティブ: {stake_info.get('active', 0) / 10**9:.5f} SOL")
    print(f"非アクティブ: {stake_info.get('inactive', 0) / 10**9:.5f} SOL")

    return stake_info

# ステーキング報酬の計算
def calculate_rewards(active_stake, apr=0.07, days=365):
    """ステーキング報酬の計算（推定）"""
    daily_apr = apr / 365
    rewards = []
    current_stake = active_stake

    for day in range(1, days + 1):
        daily_reward = current_stake * daily_apr
        current_stake += daily_reward
        rewards.append({"day": day, "total_stake": current_stake, "daily_reward": daily_reward})

    return rewards

# 報酬のグラフ表示
def plot_rewards(rewards_data):
    """ステーキング報酬の推移をグラフ表示"""
    df = pd.DataFrame(rewards_data)

    plt.figure(figsize=(12, 6))

    # 累積ステーキング量
    plt.subplot(1, 2, 1)
    plt.plot(df["day"], df["total_stake"])
    plt.title("累積ステーキング量")
    plt.xlabel("日数")
    plt.ylabel("SOL")
    plt.grid(True)

    # 日次報酬
    plt.subplot(1, 2, 2)
    plt.plot(df["day"], df["daily_reward"])
    plt.title("日次報酬")
    plt.xlabel("日数")
    plt.ylabel("SOL/日")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# メイン関数
def main():
    print("=== Solanaステーキングツール for Google Colab ===")

    # ネットワーク選択
    network = input("ネットワークを選択してください (mainnet/testnet/devnet) [mainnet]: ") or "mainnet"
    client = setup_client(network)

    # ウォレット読み込み
    wallet = load_wallet()
    if wallet is None:
        print("ウォレットの読み込みに失敗しました。終了します。")
        return

    print(f"ウォレットアドレス: {wallet.public_key}")

    # 残高確認
    balance = check_balance(client, wallet.public_key)
    print(f"残高: {balance:.5f} SOL")

    # バリデーターリスト取得
    print("バリデーターリストを取得中...")
    validators = get_validators(client)

    # バリデーター選択
    validator = select_validator(validators)
    print(f"選択されたバリデーター: {validator['identity']}")
    print(f"コミッション率: {validator['commission']}%")

    # ステーキング量の設定
    stake_amount = float(input(f"ステーキングするSOL量を入力してください (最大 {balance-0.1:.5f}): "))
    if stake_amount > balance - 0.1:
        print("残高が不足しています。手数料のために少なくとも0.1 SOL残してください。")
        return

    # シード値の設定
    seed = input("ステーキングアカウントのシード値を入力してください [stake1]: ") or "stake1"

    # ステーキング実行
    print(f"{stake_amount} SOLをステーキングします...")
    stake_account, tx_sig = create_stake_and_delegate(client, wallet, validator, stake_amount, seed)

    if stake_account:
        print(f"ステーキングアカウント: {stake_account}")
        print(f"トランザクションID: {tx_sig}")

        # 少し待機
        print("ステーキング情報を取得しています...")
        time.sleep(2)

        # ステーキング情報の取得
        stake_info = get_stake_info(client, stake_account)

        # 報酬の予測
        if stake_info and stake_info["state"] == "active":
            active_stake = stake_info["active"] / 10**9
            print("\n=== 報酬予測 ===")
            apr = (7.0 - validator["commission"] / 100)  # コミッション率を考慮した年利
            print(f"推定年利: {apr*100:.2f}%（バリデーターコミッション考慮後）")

            rewards = calculate_rewards(active_stake, apr)

            # 年間報酬の表示
            annual_reward = rewards[-1]["total_stake"] - active_stake
            print(f"1年後の推定報酬: {annual_reward:.5f} SOL")

            # グラフ表示
            plot_rewards(rewards)

        print("\nSolanaエクスプローラーで確認:")
        explorer_url = "https://explorer.solana.com"
        if network != "mainnet":
            explorer_url += f"?cluster={network}"
        print(f"ステーキングアカウント: {explorer_url}/account/{stake_account}")
        print(f"トランザクション: {explorer_url}/tx/{tx_sig}")

if __name__ == "__main__":
    main()